In [ ]:
#!pip install tensorflow==2.0.0

# CNN for Face Image Recognition

This Python code is automatic recognition of the class of a given image

In our case, the image will contain the face of some individual


## The Data

-----------

### PLEASE NOTE: THIS DATASET CONTAINS MANY CLASSES .. IT CAN BE DOWNLOADED FROM THE LINK BELOW.
### I have downloaded it and manually removed many classes (folders)
### I have also added a folder containing my own images
--------
----------
--------
ORIGINAL DATA SOURCE:

https://cswww.essex.ac.uk/mv/allfaces/faces95.html

-----------
* The data folders are 'train' and 'test' 

* Each folder contains subfolders with indiviual names (i.e. persons)

* Each indiidual folder contains face images of that individual

* The folder name is the person name (it is the class of the image)

* The Structure is like this:

* Image Data Folder
    * Person 1
        * 0.jpg
        * 1.jpg
        * ...
    * Person 2
        * 0.jpg
        * 1.jpg
        * ...
    * ...
    * Person n


* The same structure in 'train' and 'test' folders

* Train and Test images do not overlap

* Only a small number of images to save time .. you can add more on your own time

--------
----------
------------


**Note: We will be dealing with real image files, NOT numpy arrays. Which means a large part of this process will be learning how to work with and deal with large groups of image files. This is too much data to fit in memory as a numpy array, so we'll need to feed it into our model in batches. **

### Visualizing the Data


-------
Let's take a closer look at the data.

In [ ]:
import matplotlib.pyplot as plt
import cv2
# Technically not necessary in newest versions of jupyter
%matplotlib inline

In [ ]:
person1 = cv2.imread('face_images/train/noureddin/noureddin11.jpg')
person1 = cv2.cvtColor(person1,cv2.COLOR_BGR2RGB)

In [ ]:
type(person1)

In [ ]:
person1.shape

In [ ]:
plt.imshow(person1)

In [ ]:
person2 = cv2.imread('face_images/test/noureddin/noureddin18.jpg')
person2 = cv2.cvtColor(person2,cv2.COLOR_BGR2RGB)

In [ ]:
person2.shape

In [ ]:
plt.imshow(person2)

## Preparing the Data for the model

Usually your training data contains lots and lots of images .. this is too much data to read all at once in memory. We can use some built in functions in Keras to automatically process the data, generate a flow of batches from a directory, and also manipulate the images.

### Image Manipulation

Its usually a good idea to manipulate the images with rotation, resizing, and scaling so the model becomes more robust to different images that our data set doesn't have. We can use the **ImageDataGenerator** to do this automatically for us. Check out the documentation for a full list of all the parameters you can use here!

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image up to 20 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
plt.imshow(image_gen.random_transform(person1))

In [ ]:
plt.imshow(image_gen.random_transform(person2))

### Generating many manipulated images from a directory


In order to use .flow_from_directory, you must organize the images in sub-directories (and this is what I have done for you .. this is the recommended way of doing it). This is an absolute requirement, otherwise the method won't work. The directories should only contain images of one class, so one folder per class of images.

Structure Needed:

* Image Data Folder
    * Class 1
        * 0.jpg
        * 1.jpg
        * ...
    * Class 2
        * 0.jpg
        * 1.jpg
        * ...
    * ...
    * Class n

In [ ]:
image_gen.flow_from_directory('face_images/train/')

In [ ]:
image_gen.flow_from_directory('face_images/test/')

### Resizing Images

Let's have Keras resize all the images to 200 pixels by 180 pixels once they've been manipulated.

In [ ]:
# width,height,channels
image_shape = (200,180,3)

# Creating the Model

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()
# As we have seen on the slides .. we add Convolutional layers and follow them by Pooling
# Notice how we configure the number of filters, filter size etc 
# There are hyper parameters which we can experiment with!
# Check the padding parameter
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=132, kernel_size=(3,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten the feature maps to feed into a dense layer as explained on the slides
model.add(Flatten())

# and here's our dense layer
model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its multi-class so we use softmax and categorical_crossentropy
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Training the Model

In [ ]:
# you can experiment with this .. depends on data size etc
batch_size = 8

train_image_gen = image_gen.flow_from_directory('face_images/train/',
                                                #to automatically resize the images
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
train_image_gen.classes

In [ ]:
test_image_gen = image_gen.flow_from_directory('face_images/test/',
                                               #to automatically resize the images
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
train_image_gen.class_indices

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### If you run this for a large number of epochs it will take a long time
### 20 epochs on my laptop take ~27 minutes

In [ ]:
%%time
results = model.fit_generator(train_image_gen,epochs=5,
                              #this is useful when you have a large number of images
                              steps_per_epoch=150,
                              validation_data=test_image_gen,
                             validation_steps=12)

# Evaluating the Model

In [ ]:
results.history

In [ ]:
plt.plot(results.history['accuracy'])

# Predicting on new images

In [ ]:
#train_image_gen.class_indices
label_map = train_image_gen.class_indices
label_map = dict(map(reversed, label_map.items()))


label_map

In [ ]:
import numpy as np
from keras.preprocessing import image

test_file = 'face_images/test/noureddin/noureddin16.jpg'

test_img = image.load_img(test_file, target_size=(200, 180))

test_img = image.img_to_array(test_img)

test_img = np.expand_dims(test_img, axis=0)
test_img = test_img/255

In [ ]:
test_img.shape

In [ ]:
from keras.models import load_model
model = load_model('face_rec_20epochs.h5')

In [ ]:
model.predict_classes(test_img)

In [ ]:
prediction_prob = model.predict(test_img)

In [ ]:
# Output prediction
print(f'Probabilities of classes for this image: {prediction_prob} ')

In [ ]:
y_prob = model.predict(test_img)
print(y_prob)
label_map[y_prob.argmax(axis=-1)[0]]

## Now let's save the model to use it with our live cam!

In [ ]:
model.save('face_rec22.h5')

# Well done!